In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
from fuzzywuzzy import process
import numpy as np
import os
import joblib

# Load pre-trained SVC model
current_dir = os.getcwd()
svc_model_path = os.path.join(current_dir,'..','saved_model', 'svc_model.joblib')
svc_classifier = joblib.load(svc_model_path)

# Example dictionaries for label encoding (replace with actual label encoders)
symptom_labels = {'fever': 0, 'cough': 1, 'headache': 2, 'fatigue': 3}
cause_labels = {'virus': 0, 'stress': 1, 'overwork': 2}
disease_labels = {'flu': 0, 'cold': 1, 'migraine': 2, 'anemia': 3}
medicine_labels = {'antiviral': 0, 'cough syrup': 1, 'pain reliever': 2, 'iron supplement': 3}

# Initialize Tkinter
root = tk.Tk()
root.title("Medicine Prediction")

# Function to predict medicine
def predict_medicine(event=None):
    # Get input values
    symptoms = symptoms_entry.get()
    causes = causes_entry.get()
    disease = disease_entry.get()

    # Convert input into preprocessed format
    symptoms = preprocess_input(symptoms)
    causes = preprocess_input(causes)
    disease = preprocess_input(disease)

    # Perform fuzzy matching to find the closest matching labels for symptoms, causes, and disease
    closest_symptom = process.extractOne(symptoms, symptom_labels.keys())[0]
    closest_cause = process.extractOne(causes, cause_labels.keys())[0]
    closest_disease = process.extractOne(disease, disease_labels.keys())[0]

    # Convert closest matches into encoded format
    symptoms_encoded = symptom_labels.get(closest_symptom)
    causes_encoded = cause_labels.get(closest_cause)
    disease_encoded = disease_labels.get(closest_disease)

    if symptoms_encoded is None or causes_encoded is None or disease_encoded is None:
        messagebox.showerror("Error", "Unable to predict medicine. Please check your input.")
        return

    # Create feature vector for prediction
    features = np.array([[symptoms_encoded, causes_encoded, disease_encoded]])

    # Predict medicine using the pre-trained SVC classifier
    predicted_med_label = svc_classifier.predict(features)[0]

    # Find the corresponding medicine name for the predicted label
    predicted_med_name = None
    for med_name, med_label in medicine_labels.items():
        if med_label == predicted_med_label:
            predicted_med_name = med_name
            break

    # Show prediction result
    messagebox.showinfo("Prediction Result", f"Predicted medicine: {predicted_med_name}")

# Function to preprocess input
def preprocess_input(user_input):
    return user_input.lower().replace(" ", "")

# Function to move focus to the next entry field when Enter is pressed
def focus_next_entry(event, entry_field):
    entry_field.focus()
    return "break"

# GUI Layout
mainframe = ttk.Frame(root, padding="20")
mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
mainframe.columnconfigure(0, weight=1)
mainframe.rowconfigure(0, weight=1)

symptoms_label = ttk.Label(mainframe, text="Enter symptoms:")
symptoms_label.grid(row=0, column=0, sticky=tk.W, padx=5, pady=5)

symptoms_entry = ttk.Entry(mainframe)
symptoms_entry.grid(row=0, column=1, sticky=tk.W, padx=5, pady=5)
symptoms_entry.focus()

causes_label = ttk.Label(mainframe, text="Enter causes:")
causes_label.grid(row=1, column=0, sticky=tk.W, padx=5, pady=5)

causes_entry = ttk.Entry(mainframe)
causes_entry.grid(row=1, column=1, sticky=tk.W, padx=5, pady=5)

disease_label = ttk.Label(mainframe, text="Enter disease:")
disease_label.grid(row=2, column=0, sticky=tk.W, padx=5, pady=5)

disease_entry = ttk.Entry(mainframe)
disease_entry.grid(row=2, column=1, sticky=tk.W, padx=5, pady=5)

# Bind Enter key to appropriate functions for moving focus and predicting medicine
symptoms_entry.bind("<Return>", lambda event: focus_next_entry(event, causes_entry))
causes_entry.bind("<Return>", lambda event: focus_next_entry(event, disease_entry))
disease_entry.bind("<Return>", predict_medicine)

predict_button = ttk.Button(mainframe, text="Predict Medicine", command=predict_medicine)
predict_button.grid(row=3, column=0, columnspan=2, pady=10)

# Run the GUI
root.mainloop()
